<div style="width:100%; background-color: #000041"><a target="_blank" href="http://university.yugabyte.com"><img src="assets/YBU_Logo.png" /></a></div><br>

> **Migrate and Modernize with YugabyteDB Voyager**
>
> Enroll for free at [Yugabyte University](https://university.yugabyte.com/courses/YugabyteDB-Voyager).
>

<br>
This notebook file is:

`06_Migrate_to_YugabyteDB_Managed.ipynb`

## 🛠️ Requirements
Here are the requirements for this notebook:
- ✅ Create the notebook variables in `01_Lab_Setup.ipynb`, which you previously did
- ✅ Create the `ds_ybu` database, which you previously did
- ✅ Create a YugabyteDB Managed free, sandbox cluster at [https://cloud.yugabyte.com](https://cloud.yugabyte.com), *which you must do next*
- ☑️ Import the notebook variables, *which you must do next*
- ☑️ Creat *which you must do next*


### Select your notebook kernel
- In the Notebook toolbar, click **Select Kernel**.
<br>
<img width=50% src="assets/01_01_Select_Kernel_Toolbar.png" />

- Next, in the dropdown, select **Python 3.8.13** or higher.
<br>
<img width=50% src="assets/01_02_Select_Kernel_Dropdown.png" />

That's it!

## ⛑️ Getting help
The best way to get help us from the Yugabyte University team. You can post your question in YugabyteDB Community Slack in the #training or #yb-university channels. Join our [community](https://communityinviter.com/apps/yugabyte-db/register).

## 👣 Set up steps for this notebook
Here are the steps to setup this lab:

- Import the notebook variables

### 👇 Import the notebook variables

> 👉 IMPORTANT! 👈
> 
> Do **NOT** skip running the following cell. 
> 

The following Python cell reads the stored variables created in the `01_Lab_Setup.ipynb` notebook. To run the script, select Execute Cell (Play Arrow) in the left gutter of the cell. 

👇 👇 👇 

In [ ]:
# Use %store -r to read 01_Lab_Setup variables
%store -r MY_YB_PATH
%store -r MY_YB_PATH_DATA

%store -r MY_GITPOD_WORKSPACE_URL

%store -r MY_DB_NAME
%store -r MY_DB_PORT
%store -r MY_MYSQL_DB_NAME

%store -r MY_HOST_IPv4_01
%store -r MY_HOST_IPv4_02
%store -r MY_HOST_IPv4_03

%store -r MY_TSERVER_WEBSERVER_PORT
%store -r MY_MYSQL_PORT

%store -r MY_YB_MASTER_HOST_GITPOD_URL
%store -r MY_YB_TSERVER_HOST_GITPOD_URL

%store -r MY_NOTEBOOK_DIR
%store -r MY_NOTEBOOK_DATA_FOLDER
%store -r MY_NOTEBOOK_UTILS_FOLDER
%store -r MY_NOTEBOOK_VOYAGER_TMP

%store -r MY_UTIL_FUNCTIONS_FILE
%store -r MY_UTIL_YBTSERVER_METRICS_FILE

%store -r BETA_FAST_DATA_EXPORT

---
## Prepare YugabyteDB Managed
- Create a sandbox cluster
- Add this Gitpod to the IP Allow List
- Download the `root.crt` file
- Copy the DNS Host name
- Create a target database and Voyager user

### Create a sandbox cluster 

Create a YugabyteDB Managed free, sandbox cluster at [https://cloud.yugabyte.com](https://cloud.yugabyte.com).

`Cluster Name` = migrate-mysql

### Add this Gitpod to the IP Allow List
Run the following to determine the public ip address of this Gitpod host.

In [ ]:
%%bash 

MY_GITPOD_IP_ADDRESS=`curl https://ipinfo.io/ip`
echo $MY_GITPOD_IP_ADDRESS

> No IP Address ???
>
> If you don't get an IP address, rerun the above again! It may make a few times.

In the YugabyteDB Managed console, in Networking, select **Add IP Address**.

- `IP ALLOW LIST NAME` = gitpod
- `DESCRIPTION` = gitpod
- `IP ADDRESS` = Add Current IP Address , MY_GITPOD_IP_ADDRESS


In Clusters, Actions, select `Add IP Allow List``.

Next, select `Add Existing IP Allow List`.

In the dialog, select `gitpod`.

Select Confirm.

### Download the `root.crt` file

Select Connect.

In Connect to Cluster, select Connect to your Application.

Select `Download CA Cert`.

Copy this to your Gitpod VS Code environment into `\tmp`. You can drag and drop.

> 👉 IMPORTANT! 👈
> - Copy your `root.crt` to the `\tmp` directory in the file explorer of the Gitpod environment. You can literally drag and drop the file to the `\tmp` in the Explore panel. There is file already there that you can ignore, `nada.txt`.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}
echo $NOTEBOOK_VOYAGER_TMP
cat  $NOTEBOOK_VOYAGER_TMP/root.crt

### Copy the DNS Host name

For YSQL, select Parameters.

Copy Host, for example: 

`us-east-1.fb67380f-e7d8-4709-8fe0-5a7182146558.aws.ybdb.io`


> 👉 IMPORTANT! 👈
>
> Make sure that your copy the host DNS name for your YugabyteDB Managed sandbox cluster.

### Create a target database and Voyager user

Create the target database in your YugabyteDB cluster. The database name can be the same or different from the source database name. If the target database name is not provided, yb-voyager assumes the target database name to be yugabyte. If you do not want to import in the default yugabyte database, specify the name of the target database name using the --target-db-name argument to the yb-voyager import commands.

```
CREATE DATABASE target_db_name;
```
```
CREATE USER ybvoyager PASSWORD 'password';
GRANT yb_superuser TO ybvoyager;
```


> 👉 IMPORTANT! 👈
> - Change the values in the variables for `ADD_HOST_HERE`, `ADD_USER_HERE`, `ADD_PASSWORD_HERE` 
> - Copy your `root.crt` to the `\tmp` directory in the file explorer

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_NOTEBOOK_VOYAGER_TMP" # create database
YB_PATH=${1}
DB_NAME=${2}
NOTEBOOK_VOYAGER_TMP=${3}

cd $YB_PATH
echo $YB_PATH
echo $NOTEBOOK_VOYAGER_TMP

## IMPORTNAT: You must update these variables
MY_YUGABYTEDB_MANAGED_HOST=ADD_HOST_HERE
MY_YUGABYTEDB_MANAGED_YSQL_USER=ADD_USER_HERE
MY_YUGABYTEDB_MANAGED_YSQL_PASSWORD=ADD_PASSWORD_HERE


MY_YUGABYTEDB_MANAGED_PORT=5433
MY_YUGABYTEDB_MANAGED_HOST_PORT=${MY_YUGABYTEDB_MANAGED_HOST}:${MY_YUGABYTEDB_MANAGED_PORT}
MY_YUGABYTEDB_MANAGED_CRT=${NOTEBOOK_VOYAGER_TMP}/root.crt
MY_YUGABYTEDB_MANAGED_CONN="postgresql://${MY_YUGABYTEDB_MANAGED_YSQL_USER}:${MY_YUGABYTEDB_MANAGED_YSQL_PASSWORD}@${MY_YUGABYTEDB_MANAGED_HOST_PORT}/yugabyte?ssl=true&sslmode=verify-full&sslrootcert=${MY_YUGABYTEDB_MANAGED_CRT}"
echo $MY_YUGABYTEDB_MANAGED_CONN

./bin/ysqlsh $MY_YUGABYTEDB_MANAGED_CONN -c "drop database if exists "${DB_NAME}";" 
./bin/ysqlsh $MY_YUGABYTEDB_MANAGED_CONN -c "create database "${DB_NAME}";" 
./bin/ysqlsh $MY_YUGABYTEDB_MANAGED_CONN -c "create user ybvoyager PASSWORD 'Yugabyte#1'; GRANT yb_superuser TO ybvoyager;" 


Verify that the `ybvoyager` user can connect.

> 👉 IMPORTANT! 👈
> - Change the values in the variables for `ADD_HOST_HERE` 
> - Copy your `root.crt` to the `\tmp` directory in the file explorer

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_NOTEBOOK_VOYAGER_TMP" # create database
YB_PATH=${1}
DB_NAME=${2}
NOTEBOOK_VOYAGER_TMP=${3}

cd $YB_PATH
echo $YB_PATH
echo $NOTEBOOK_VOYAGER_TMP

## IMPORTNAT: You must update these variables
MY_YUGABYTEDB_MANAGED_HOST=ADD_HOST_HERE
MY_YUGABYTEDB_MANAGED_YSQL_USER=ybvoyager
MY_YUGABYTEDB_MANAGED_YSQL_PASSWORD=Yugabyte#1


MY_YUGABYTEDB_MANAGED_PORT=5433
MY_YUGABYTEDB_MANAGED_HOST_PORT=${MY_YUGABYTEDB_MANAGED_HOST}:${MY_YUGABYTEDB_MANAGED_PORT}
MY_YUGABYTEDB_MANAGED_CRT=${NOTEBOOK_VOYAGER_TMP}/root.crt
MY_YUGABYTEDB_MANAGED_CONN="postgresql://${MY_YUGABYTEDB_MANAGED_YSQL_USER}:${MY_YUGABYTEDB_MANAGED_YSQL_PASSWORD}@${MY_YUGABYTEDB_MANAGED_HOST_PORT}/yugabyte?ssl=true&sslmode=verify-full&sslrootcert=${MY_YUGABYTEDB_MANAGED_CRT}"
echo $MY_YUGABYTEDB_MANAGED_CONN

./bin/ysqlsh $MY_YUGABYTEDB_MANAGED_CONN -c "select 1;" 

---
## Import modified schema

> 👉 IMPORTANT! 👈
> - Change the values in the variables for `ADD_HOST_HERE`
> - Copy your `root.crt` to the `\tmp` directory in the file explorer

In [ ]:
%%bash -s "$MY_DB_NAME"  "$MY_DB_PORT" "$MY_HOST_IPv4_01" "$MY_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}

## IMPORTNAT: You must update these variables
MY_YUGABYTEDB_MANAGED_HOST=ADD_HOST_HERE
MY_YUGABYTEDB_MANAGED_YSQL_USER=ybvoyager
MY_YUGABYTEDB_MANAGED_YSQL_PASSWORD=Yugabyte#1


MY_YUGABYTEDB_MANAGED_PORT=5433
MY_YUGABYTEDB_MANAGED_HOST_PORT=${MY_YUGABYTEDB_MANAGED_HOST}:${MY_YUGABYTEDB_MANAGED_PORT}
MY_YUGABYTEDB_MANAGED_CRT=${NOTEBOOK_VOYAGER_TMP}/root.crt
MY_YUGABYTEDB_MANAGED_CONN="postgresql://${MY_YUGABYTEDB_MANAGED_YSQL_USER}:${MY_YUGABYTEDB_MANAGED_YSQL_PASSWORD}@${MY_YUGABYTEDB_MANAGED_HOST_PORT}/yugabyte?ssl=true&sslmode=verify-full&sslrootcert=${MY_YUGABYTEDB_MANAGED_CRT}"
echo $MY_YUGABYTEDB_MANAGED_CONN


yb-voyager import schema  -y  \
  --start-clean \
  --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir \
  --target-db-host ${MY_YUGABYTEDB_MANAGED_HOST} \
  --target-db-port  ${MY_YUGABYTEDB_MANAGED_PORT} \
  --target-db-name  ${DB_NAME} \
  --target-db-user ${MY_YUGABYTEDB_MANAGED_YSQL_USER} \
  --target-db-password ${MY_YUGABYTEDB_MANAGED_YSQL_PASSWORD} \
  --target-ssl-mode "verify-full" \
  --target-ssl-root-cert ${MY_YUGABYTEDB_MANAGED_CRT} \
  --target-db-schema "sakila" > /dev/null
  

#### Sanity check

> 👉 IMPORTANT! 👈
> - Change the values in the variables for `ADD_HOST_HERE`
> - Copy your `root.crt` to the `\tmp` directory in the file explorer

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_NOTEBOOK_VOYAGER_TMP" # create database
YB_PATH=${1}
DB_NAME=${2}
NOTEBOOK_VOYAGER_TMP=${3}

cd $YB_PATH
echo $YB_PATH
echo $NOTEBOOK_VOYAGER_TMP

## IMPORTNAT: You must update these variables
MY_YUGABYTEDB_MANAGED_HOST=ADD_HOST_HERE
MY_YUGABYTEDB_MANAGED_YSQL_USER=ybvoyager
MY_YUGABYTEDB_MANAGED_YSQL_PASSWORD=Yugabyte#1


MY_YUGABYTEDB_MANAGED_PORT=5433
MY_YUGABYTEDB_MANAGED_HOST_PORT=${MY_YUGABYTEDB_MANAGED_HOST}:${MY_YUGABYTEDB_MANAGED_PORT}
MY_YUGABYTEDB_MANAGED_CRT=${NOTEBOOK_VOYAGER_TMP}/root.crt
MY_YUGABYTEDB_MANAGED_CONN="postgresql://${MY_YUGABYTEDB_MANAGED_YSQL_USER}:${MY_YUGABYTEDB_MANAGED_YSQL_PASSWORD}@${MY_YUGABYTEDB_MANAGED_HOST_PORT}/${DB_NAME}?ssl=true&sslmode=verify-full&sslrootcert=${MY_YUGABYTEDB_MANAGED_CRT}"
echo $MY_YUGABYTEDB_MANAGED_CONN

./bin/ysqlsh $MY_YUGABYTEDB_MANAGED_CONN -c "\dn;" 


---
## Import source data
Import the source data to the target YugabyteDB database using the `yb-voyager import data` command.

> 👉 IMPORTANT! 👈
> - Change the values in the variables for `ADD_HOST_HERE`
> - Copy your `root.crt` to the `\tmp` directory in the file explorer

In [ ]:
%%bash -s "$MY_DB_NAME"  "$MY_DB_PORT" "$MY_HOST_IPv4_01" "$MY_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}


## IMPORTNAT: You must update these variables
MY_YUGABYTEDB_MANAGED_HOST=ADD_HOST_HERE
MY_YUGABYTEDB_MANAGED_YSQL_USER=ybvoyager
MY_YUGABYTEDB_MANAGED_YSQL_PASSWORD=Yugabyte#1


MY_YUGABYTEDB_MANAGED_PORT=5433
MY_YUGABYTEDB_MANAGED_HOST_PORT=${MY_YUGABYTEDB_MANAGED_HOST}:${MY_YUGABYTEDB_MANAGED_PORT}
MY_YUGABYTEDB_MANAGED_CRT=${NOTEBOOK_VOYAGER_TMP}/root.crt
MY_YUGABYTEDB_MANAGED_CONN="postgresql://${MY_YUGABYTEDB_MANAGED_YSQL_USER}:${MY_YUGABYTEDB_MANAGED_YSQL_PASSWORD}@${MY_YUGABYTEDB_MANAGED_HOST_PORT}/${DB_NAME}?ssl=true&sslmode=verify-full&sslrootcert=${MY_YUGABYTEDB_MANAGED_CRT}"
echo $MY_YUGABYTEDB_MANAGED_CONN


##update to use connection string, user, and SSL?
yb-voyager import data  --start-clean  -y \
  --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir \
  --target-db-host ${MY_YUGABYTEDB_MANAGED_HOST} \
  --target-db-port  ${MY_YUGABYTEDB_MANAGED_PORT} \
  --target-db-name  ${DB_NAME} \
  --target-db-user ${MY_YUGABYTEDB_MANAGED_YSQL_USER} \
  --target-db-password ${MY_YUGABYTEDB_MANAGED_YSQL_PASSWORD} \
  --target-ssl-mode "verify-full" \
  --target-ssl-root-cert ${MY_YUGABYTEDB_MANAGED_CRT} \
  --target-db-schema "sakila" > /dev/null

You can check the status of the import.

In [ ]:
%%bash -s "$MY_MYSQL_DB_NAME"  "$MY_MYSQL_PORT" "$MY_HOST_IPv4_01" "$MY_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}

yb-voyager import data status --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir 


---
## Post Import Data
After importing the data, rerun the import schema command, but this time, using the arguments, `post-import-data` and `refresh-mviews`.


> 👉 IMPORTANT! 👈
> - Change the values in the variables for `ADD_HOST_HERE` 
> - Copy your `root.crt` to the `\tmp` directory in the file explorer


In [ ]:
%%bash -s "$MY_DB_NAME"  "$MY_DB_PORT" "$MY_HOST_IPv4_01" "$MY_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}


## IMPORTNAT: You must update these variables
MY_YUGABYTEDB_MANAGED_HOST=ADD_HOST_HERE
MY_YUGABYTEDB_MANAGED_YSQL_USER=ybvoyager
MY_YUGABYTEDB_MANAGED_YSQL_PASSWORD=Yugabyte#1


MY_YUGABYTEDB_MANAGED_PORT=5433
MY_YUGABYTEDB_MANAGED_HOST_PORT=${MY_YUGABYTEDB_MANAGED_HOST}:${MY_YUGABYTEDB_MANAGED_PORT}
MY_YUGABYTEDB_MANAGED_CRT=${NOTEBOOK_VOYAGER_TMP}/root.crt
MY_YUGABYTEDB_MANAGED_CONN="postgresql://${MY_YUGABYTEDB_MANAGED_YSQL_USER}:${MY_YUGABYTEDB_MANAGED_YSQL_PASSWORD}@${MY_YUGABYTEDB_MANAGED_HOST_PORT}/${DB_NAME}?ssl=true&sslmode=verify-full&sslrootcert=${MY_YUGABYTEDB_MANAGED_CRT}"
echo $MY_YUGABYTEDB_MANAGED_CONN


##update to use connection string, user, and SSL?
yb-voyager import schema  -y \
  --post-import-data \
  --refresh-mviews \
  --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir \
  --target-db-host ${MY_YUGABYTEDB_MANAGED_HOST} \
  --target-db-port  ${MY_YUGABYTEDB_MANAGED_PORT} \
  --target-db-name  ${DB_NAME} \
  --target-db-user ${MY_YUGABYTEDB_MANAGED_YSQL_USER} \
  --target-db-password ${MY_YUGABYTEDB_MANAGED_YSQL_PASSWORD} \
  --target-ssl-mode "verify-full" \
  --target-ssl-root-cert ${MY_YUGABYTEDB_MANAGED_CRT} \
  --target-db-schema "sakila" > /dev/null

---
## Verify migration
After completing the migration of the schema and the data,the automated migration is finished. Next is the manual verification of the migration success. 

You need to manually validate the successful migration of database objects, including the post import data objects such as triggers and indexes.

You also need to manually run validation queries on both the source and target database to ensure that the data is correctly migrated. A sample query to validate the databases can include checking the row count of each table.


### Connect to your cluster in YugabyteDB Managed using the PostgreSQL Driver for Python

> 👉 IMPORTANT! 👈
> - Change the values in the variables for `ADD_HOST_HERE`.

Copy the connection string from the above cell output into `conn_str`

In [ ]:
# Connect to db_ybu in YugabyteDB Managed, requires you to update `ADD_HOST_HERE`

import psycopg2
import sqlalchemy as alc
from sqlalchemy import create_engine


db_name=MY_DB_NAME

managed_host='ADD_HOST_HERE'

connection_managed_str='postgresql://ybvoyager:Yugabyte#1@'+managed_host+':5433/'+db_name
print(connection_managed_str)
engine = create_engine(connection_managed_str)

#### Load SQL magic extension
>IMPORTANT!
>
> To use SQL magic, you must run the following cell that loads the notebook extension.

In [ ]:
%reload_ext sql
# creates connection for sql magic
%sql {connection_managed_str}

### Run queries to validate objects

Verify the creation of indexes.

In [ ]:
%%sql
select tablename, indexname, indexdef from pg_indexes where schemaname = 'sakila' order by tablename, indexname;

Verify the creation of triggers.

In [ ]:
%%sql
select  event_object_table AS table_name, trigger_name from information_schema.triggers where trigger_schema = 'sakila' group by table_name, trigger_name order by table_name, trigger_name;

### Run queries

In [ ]:
%%sql
select * from sakila.actor where UPPER(first_name) = 'SCARLETT';

In [ ]:
%%sql
select count(distinct last_name) from sakila.actor;

In [ ]:
%%sql
select actor.actor_id, actor.first_name, actor.last_name,
       count(actor_id) as film_count
from sakila.actor join sakila.film_actor using (actor_id)
group by actor_id
order by film_count desc
limit 1;

---

## 😊 Next up!
Continue your learning by opening the next notebook, `07_Demystifying_Table_Sharding.ipynb`. 

Or, to open the notebook from GitPod, run the following:

In [ ]:
%%bash
gp open '07_Demystifying_Table_Sharding.ipynb'